In [1]:
import sys
sys.path.append("../../../../../")
import tempfile
from pathlib import Path


document = { 
    "regex_key_one": "value", 
    "regex_key_two": "*1value2*" 
}

rule_yaml = """
---
filter: 'regex_key_one: "/.*value.*/" AND regex_key_two: "/.*value.*/"'
regex_fields:
  - "regex_key_one"
  - "regex_key_two"
pseudonymizer:
  mapping:
      regex_key_one: "RE_WHOLE_FIELD"
      regex_key_two: "RE_WHOLE_FIELD"
  description: "..."
"""



document = { 
    "regex_key_one": "*value*", 
    "regex_key_two": "value", 
    "test_pseudonymizer": "test", "something_special": "pseudonymize_me"
}

rule_yaml = """
---
filter: "test_pseudonymizer AND something_special"
pseudonymizer:
  id: pseudonymizer-1352bc0a-53ae-4740-bb9e-1e865f63375f
  mapping:
    something_special: "RE_WHOLE_FIELD"
description: "..."
"""




rule_path = Path(tempfile.gettempdir()) / "pseudonymizer"
rule_path.mkdir(exist_ok=True)
rule_file = rule_path / "data-stream.yml"
rule_file.write_text(rule_yaml)

processor_config = {
    "mypseudonymizer": {
        "type": "pseudonymizer",
        "specific_rules": [str(rule_path)],
        "generic_rules": ["../../../../../examples/exampledata/rules/pseudonymizer/generic/"],
        "outputs": [{"kafka": "topic"}],
        "pubkey_analyst": "../../../../../tests/testdata/unit/pseudonymizer/example_analyst_pub.pem",
        "pubkey_depseudo": "../../../../../tests/testdata/unit/pseudonymizer/example_depseudo_pub.pem",
        "hash_salt": "a_secret_tasty_ingredient",
        "regex_mapping":  "../../../../../tests/testdata/unit/pseudonymizer/rules/regex_mapping.yml",
        "max_cached_pseudonyms": 1000000
    }
}

In [2]:
from unittest import mock
from logprep.factory import Factory

mock_logger = mock.MagicMock()
pseudonymizer_processor = Factory.create(processor_config)
pseudonymizer_processor

pseudonymizer

In [3]:
from copy import deepcopy
mydocument = deepcopy(document)

print(f"before: {mydocument}")
pseudonymizer_processor.process(mydocument)
print(f"after : {mydocument}")

before: {'regex_key_one': '*value*', 'regex_key_two': 'value', 'test_pseudonymizer': 'test', 'something_special': 'pseudonymize_me'}


ProcessingCriticalError: ProcessingCriticalError: ProcessingCriticalError: 'str' object has no attribute 'groups' -> event was send to error output and further processing stopped, rule.id='pseudonymizer-1352bc0a-53ae-4740-bb9e-1e865f63375f', rule.description='...', event={'regex_key_one': '*value*', 'regex_key_two': 'value', 'test_pseudonymizer': 'test', 'something_special': 'pseudonymize_me'}